In [1]:
path_to_files = input('Please insert the path to the pvplants folders:')

Please insert the path to the pvplants folders:/media/spiros/MY_DATA/pvplants


In [2]:
# path_to_files = input('Please insert the path to the pvplants folders:')
from usefull_funcs import *
import pickle
with open ('all_data.pickle','rb') as file:
    all_data = pickle.load(file)

# plants=list_dir(path_to_files)
# all_data=[]
# for plant in plants:
#     data = parse_ods(plant,path_to_files)
#     all_data.append({plant:data})

ppc_plants_data = []
plants_data = []
for plant in all_data:
    pl = list(plant.keys())[0]
    ppc = find_if_ppc(plant[pl])
    inv = find_inv(plant[pl])
    country = find_country(plant[pl])
    grid = find_grid_code(plant[pl]) or 'None'
    ips = find_plant_ips(path_to_files,plant[pl],pl)
    try:
        ppc_c = 'c_'+str(int(find_ppc_controller(plant[pl]))-1)
    except:
        ppc_c = '-'
    try:
        ppc_ip = [ el['address'] for el in ips['controllers'] if el['ppc_ip']=="YES"] or ['-']
    except:
        ppc_ip = ['-']
    try:
        rest_ips = [ el['address'] for el in ips['controllers'] if el['ppc_ip']=="NO"]
    except:
        rest_ips = []
    if ppc:
        ppc_meter = find_PPC_meter(plant[pl])
        ppc_nom_power = find_ppc_nom_power(plant[pl])                           
    else:
        ppc_meter = ['-','-']
        ppc_nom_power = '-'
    if ppc:
        ppc_plants_data.append({'Name':pl,
                               'PPC Meter':ppc_meter,
                               'Inverter':inv,
                               'Country':country,
                               'Grid_Code':grid,
                               'PPC Controller':ppc_c,
                               'Portal Name':find_portal_name(plant[pl]),
                               'PPC Nominal Power': ppc_nom_power,
                               '#_Controllers': ips['ncontrollers'],
                               'PPC ip': ppc_ip,
                               'Rest ips':rest_ips})
    plants_data.append({'Name':pl,
                       'Inverter':inv,
                       'PPC Meter':ppc_meter,
                       'Country':country,
                       'Grid_Code':grid,
                       'PPC Controller':ppc_c,
                       'Portal Name':find_portal_name(plant[pl]),
                       'PPC Nominal Power': ppc_nom_power,
                       '#_Controllers': ips['ncontrollers'],
                       'PPC ip': ppc_ip,
                       'Rest ips':rest_ips})

In [3]:
import sqlite3
conn = sqlite3.connect('all_plants.sqlite')
cur = conn.cursor()

In [4]:
cur.executescript('''
CREATE TABLE IF NOT EXISTS PLANTS (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    plant TEXT UNIQUE,
    portal_name TEXT,
    country_id INTEGER,
    meter_id INTEGER,
    inverter_id INTEGER,
    grid_code_id INTEGER,
    ppc_ip_id INTEGER,
    n_controllers INTEGER,
    ppc_nominal_power INTEGER,
    ppc_controller TEXT
);
CREATE TABLE IF NOT EXISTS COUNTRIES (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    country TEXT UNIQUE
);
CREATE TABLE IF NOT EXISTS IPS (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    ip TEXT UNIQUE,
    plant_id INTEGER
);
CREATE TABLE IF NOT EXISTS PPC_METERS(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    model TEXT UNIQUE,
    man_id INTEGER
);
CREATE TABLE IF NOT EXISTS METER_MANUFACTURERS(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    manufacturer TEXT UNIQUE
);
CREATE TABLE IF NOT EXISTS INVERTERS(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    model TEXT UNIQUE,
    man_id INTEGER
);
CREATE TABLE IF NOT EXISTS INVERTER_MANUFACTURERS(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    manufacturer TEXT UNIQUE
);
CREATE TABLE IF NOT EXISTS GRID_CODES(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    grid_code TEXT UNIQUE
)

''')

In [11]:
conn.commit()

In [10]:
for entry in plants_data:
    
    cur.execute('''INSERT OR IGNORE INTO PLANTS (plant) VALUES (?)''',(entry['Name'],))
    cur.execute('''SELECT id FROM PLANTS WHERE plant = ?''',(entry['Name'],))
    plant_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO COUNTRIES (country) VALUES (?)''', (entry['Country'],))
    cur.execute('''SELECT id FROM COUNTRIES WHERE country = ?''',(entry['Country'],))
    country_id = cur.fetchone()[0]
    
    
    cur.execute('''INSERT OR IGNORE INTO METER_MANUFACTURERS (manufacturer)  VALUES (?)''', (entry['PPC Meter'][0],))
    cur.execute('''SELECT id FROM METER_MANUFACTURERS WHERE manufacturer = ?''',(entry['PPC Meter'][0],))
    meter_man_id = cur.fetchone()[0]
    
    
    cur.execute('''INSERT OR IGNORE INTO PPC_METERS (model) VALUES (?)''', (entry['PPC Meter'][1],))
    cur.execute('''SELECT id FROM PPC_METERS WHERE model = ?''',(entry['PPC Meter'][1],))
    meter_id = cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO INVERTER_MANUFACTURERS (manufacturer) VALUES (?)''', (entry['Inverter'][0][0],))
    cur.execute('''SELECT id FROM INVERTER_MANUFACTURERS WHERE manufacturer = ?''',(entry['Inverter'][0][0],))
    inv_man_id = cur.fetchone()[0]
    
    
    cur.execute('''INSERT OR IGNORE INTO INVERTERS (model) VALUES (?)''', (entry['Inverter'][1][0],))
    cur.execute('''SELECT id FROM INVERTERS WHERE model = ?''',(entry['Inverter'][1][0],))
    inv_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO GRID_CODES (grid_code) VALUES (?)''', (entry['Grid_Code'],))
    cur.execute('''SELECT id FROM GRID_CODES WHERE grid_code = ?''',(entry['Grid_Code'],))
    grid_id = cur.fetchone()[0]
    
    for ip in entry['PPC ip']+entry['Rest ips']:
         cur.execute('''INSERT OR IGNORE INTO IPS (ip) VALUES (?)''', (ip,))
         cur.execute(''' UPDATE IPS SET plant_id = ? WHERE ip = ?''', (plant_id,ip))
        
    if entry['PPC ip'] != ['-']:
        cur.execute(''' SELECT id FROM IPS WHERE ip = ?''',(entry['PPC ip'][0],))
        cur.execute('''SELECT id FROM IPS WHERE ip = ?''',(entry['PPC ip'][0],))
        ip_id = cur.fetchone()[0]
    else:
        ip_id = '-'
    
    
    cur.execute(''' UPDATE PLANTS SET portal_name = ?, country_id = ?, meter_id = ?,
                        inverter_id = ?, grid_code_id = ?, ppc_ip_id = ?,
                        n_controllers = ?, ppc_nominal_power = ?, ppc_controller = ? WHERE plant = ?''',
               (entry['Portal Name'], country_id, meter_id, inv_id, grid_id, ip_id, entry['#_Controllers'],
               entry['PPC Nominal Power'],entry['PPC Controller'],entry['Name'],))
    
    cur.execute(''' UPDATE INVERTERS SET man_id = ? WHERE id = ?''', (inv_man_id,inv_id))
    
    cur.execute(''' UPDATE PPC_METERS SET man_id = ? WHERE id = ? ''', (meter_man_id,meter_id))

In [14]:
entry

{'Name': 'Anemos_Mali_Madi',
 'Inverter': ([], []),
 'PPC Meter': ['-', '-'],
 'Country': 'Greece',
 'Grid_Code': 'None',
 'PPC Controller': '-',
 'Portal Name': 'Anemos_Mali_Madi',
 'PPC Nominal Power': '-',
 '#_Controllers': 1,
 'PPC ip': ['-'],
 'Rest ips': ['172.17.17.178']}

In [15]:
data = parse_ods('Anemos_Mali_Madi',path_to_files)

Anemos_Mali_Madi



In [17]:
man = ['None']
inv = ['None']
inv =  list(set(data['PV array']['Inverter']['Model']))

In [20]:
inv.is

AttributeError: 'list' object has no attribute 'isempty'

In [19]:
not inv

True

In [129]:
man

[None]

In [130]:
man == [None]

True

In [12]:
a= None

In [8]:
b = [None, 1]


In [14]:
a == b

False

In [18]:
b.index(None)

ValueError: None is not in list

In [9]:
b

[None, 1]